# Embeddings con LSTM



In [78]:
import torch
import torch.nn as nn
from collections import Counter
from random import choices, sample, seed
from itertools import chain
from pprint import pprint

In [2]:
cd ..

c:\Users\Usuario.000\Documents\Facultad\Git\2020-2\APIT-2020-2\ProyectoFinal


In [3]:
from helpers.boletines import get_sentences

In [4]:
cd Embeddings/

c:\Users\Usuario.000\Documents\Facultad\Git\2020-2\APIT-2020-2\ProyectoFinal\Embeddings


In [29]:
SEED = 42069
seed(SEED)

In [5]:
uri = open('./../mongo_uri.txt', 'r', encoding='utf-8').read()

In [6]:
sentences = get_sentences(uri)

JAMK
1. José_Antonio_Meade fue recibido en Culiacán por cientos de simpatizantes sinaloenses .
377. El aspirante presidencial se comprometió a mejorar la infraestructura carretera e hidráulica en la entidad .
753. “ Si tuviera que resumir en una idea el país por el que apuesto para los próximos seis años , lo resumiría en tres palabras : un México líder ” , indicó .
1129. De_acuerdo_con diversas estimaciones , las armas en poder de la delincuencia organizada en México podrían alcanzar el número de 1.5 millones de piezas , esto es 3 veces el arsenal con el que cuenta , por_ejemplo , el ejército de Guatemala .
1505. Expresó que , sin_ninguna_duda y con toda certeza , logrará la victoria el 1_de_julio .
1881. Dijo que con Mikel_Arriola va por una Ciudad_de_México donde abramos la llave y salga agua , que no sea extorsionada por delegados que condicionan el agua a la militancia política , una ciudad que no tandee el agua , donde el derecho a el agua sea vigente , se eviten las fugas y se e

Desenvuelvo los grupos de sentencias en un único arreglo con todas las sentencias de todos los candidatos.

In [20]:
corpus = list(chain(*sentences.values()))

Paso los tokens a minúsculas para reducir el tamaño del vocabulario

In [35]:
corpus = [[w.lower() for w in sent] for sent in corpus]

150 oraciones de prueba para el modelo

In [56]:
k = 1000
test_sents = sample(corpus, k=k)
print('\n'.join([' '.join(sent) for sent in test_sents[::k//10]]))

eso lo distingue de ricardo_anaya , sobre quien pesan acusaciones y señalamientos muy graves , las cuales han sido formuladas , no_sólo por opositores , sino por personas confesas en procesos judiciales e , incluso , por prominentes miembros de su propio partido .
el candidato de la coalición todos_por_méxico a la presidencia_de_la_república , josé_antonio_meade , sostuvo esta tarde que morelos volverá a ser tierra de oportunidades y recuperará la paz .
apoyado en el trabajo de la estructura y con el apoyo de los ciudadanos mexicanos que sabemos que estamos a tiempo de elegir a la mejor opción para méxico , josé_antonio_meade irá de frente por la victoria el próximo_1_de_julio .
el candidato de la coalición_por_méxico a el frente_a_la_presidencia_de_la_república , ricardo_anaya_cortés , dialogó con el gobernador de chihuahua , javier_corral , a quien definió como “ un ejemplo de combate frontal a la corrupción ” .
pidió a todos los marinos y a los soldados de méxico que voten libre , p

Cuento los tokens en todas las oraciones e imprimo los 40 más frecuentes.

Agrego un diccionario para pasar de palabra a índice numérico

In [57]:
freq = sum([Counter(sent) for sent in test_sents], Counter())
print(freq.most_common()[:-40:-1])

[('observa', 1), ('presumir', 1), ('efecto', 1), ('sentimiento', 1), ('generado', 1), ('expresiones', 1), ('escuchados', 1), ('2017-2018', 1), ('proceso_electoral_federal', 1), ('consejo_general_de_el_ine', 1), ('compañeros', 1), ('amenazar', 1), ('cercanos', 1), ('escuchado', 1), ('asevera', 1), ('cantidad', 1), ('inverosímil', 1), ('60_millones_de_pesos', 1), ('reportó', 1), ('instituto', 1), ('eduardo_aguilar', 1), ('respecto', 1), ('jueves', 1), ('centrar', 1), ('construya', 1), ('35_mil', 1), ('sumaron', 1), ('empresarios_por_meade', 1), ('coctel', 1), ('asistió', 1), ('presidencia_de_la_república_de_el_pri', 1), ('vialidades', 1), ('rieles', 1), ('rené_juárez_cisneros', 1), ('funcionando', 1), ('14', 1), ('llevan', 1), ('cendis', 1), ('4_mil', 1)]


In [64]:
num_tokens = len(freq.keys())
num_tokens

5138

In [62]:
w_to_index = {
    w : ix
    for ix, (w, freq) in enumerate(freq.most_common())
    if freq > 1 # No toma en cuenta los hapax
}

In [65]:
BOS = '<BOS>'
EOS = '<EOS>'
UNK = '<UNK>'

ixBOS = num_tokens + 1
ixEOS = num_tokens + 2
ixUNK = num_tokens + 3

In [66]:
w_to_index[BOS] = ixBOS
w_to_index[EOS] = ixEOS
w_to_index[UNK] = ixUNK

Creo el diccionario inverso, para convertir de índices a palabras

In [71]:
index_to_w = [ w for w, ix in w_to_index.items() ]
len(index_to_w)

2272

In [74]:
tests = sample(w_to_index.keys(), k=10)
for w in tests:
    ix = w_to_index[w]
    assert index_to_w[ix] == w

Defino las variables para la red neuronal

In [76]:
D_out = D_in = len(index_to_w)
D_emb = 32
D_lstm = 16

In [77]:
dtype = torch.float
device = torch.device("cuda:0")

NameError: name 'torch' is not defined

In [4]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

In [5]:
# Create random Tensors to hold input and outputs.
# Setting requires_grad=False indicates that we do not need to compute gradients
# with respect to these Tensors during the backward pass.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

In [6]:
# Create random Tensors for weights.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

In [7]:
learning_rate = 1e-6

In [8]:
for t in range(500):
    # Forward pass: compute predicted y using operations on Tensors; these
    # are exactly the same operations we used to compute the forward pass using
    # Tensors, but we do not need to keep references to intermediate values since
    # we are not implementing the backward pass by hand.
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call w1.grad and w2.grad will be Tensors holding the gradient
    # of the loss with respect to w1 and w2 respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    # An alternative way is to operate on weight.data and weight.grad.data.
    # Recall that tensor.data gives a tensor that shares the storage with
    # tensor, but doesn't track history.
    # You can also use torch.optim.SGD to achieve this.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()

99 684.3082275390625
199 4.297140598297119
299 0.04988696426153183
399 0.001059292582795024
499 0.00012105940550100058
